In [27]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
import pandas as pd
import math
import openpyxl

def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    
    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
#     df[df.duplicated(subset=['ID'],keep=False)].to_excel('duplicated_화장품/미용.xlsx')
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])#'브랜드'
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [29]:
df, merged_df, attributes = makeCategoryDF(100000921)

In [30]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,헤어케어세트,샴푸,...,스틱형,쉐도우형,쿠션형,브러쉬형,새치케어,모발탄력,"린스,컨디셔너","시트,티슈형",보색린스,모발보호
0,32792384618,샴푸바 S19 바오밥 3개 + 노워시 트리트먼트 바 1개,화장품/미용,헤어케어,헤어케어세트,톤28,톤28,"[헤어케어세트, 샴푸, 린스, 트리트먼트, 머릿결개선, 향, 촉촉함(수분공급)]",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",False,True,...,False,False,False,False,False,False,False,False,False,False
2,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",False,True,...,False,False,False,False,False,False,False,False,False,False
3,40292201619,바이오틴 손상모발개선 탈모완화 샴푸 1000ml,화장품/미용,헤어케어,탈모케어,코스맥스,바이오가,"[탈모케어, 모든두피용, 모든 모발용, 일반샴푸, 탈모샴푸, 펌프형, 저자극, 세정...",False,True,...,False,False,False,False,False,False,False,False,False,False
4,5765532401,컬링 에센스 2X 150ml,화장품/미용,헤어케어,헤어에센스,아모스,아모스,"[헤어에센스, 촉촉함(수분공급), 윤기부여, 영양공급, 튜브형]",False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8075,39429418618,아쿠아본드 미네랄 마스크 250ml,화장품/미용,헤어케어,헤어팩,,,"[헤어팩, 모든 모발용, 머릿결개선, 촉촉함(수분공급), 영양공급, 모발영양, 모든...",False,False,...,False,False,False,False,False,False,False,False,False,False
8076,33316572618,아르간 헤어 에센스 100ml,화장품/미용,헤어케어,헤어에센스,세레코,셀해피코,"[헤어에센스, 모든 모발용, 펌프형, 머릿결개선, 향, 촉촉함(수분공급), 윤기부여...",False,False,...,False,False,False,False,False,False,False,False,False,False
8077,27788361531,플로랜드 프리미엄 실크 케라틴 샴푸 530ml,화장품/미용,헤어케어,샴푸,나우코스,,"[샴푸, 펌프형, 풍부한 거품, 촉촉함(수분공급), 각질케어, 약산성, 손상케어, ...",False,True,...,False,False,False,False,False,False,False,False,False,False
8078,30789191630,헤어 리페어 비타민 트리트먼트 200ml,화장품/미용,헤어케어,트리트먼트,서울화장품,정직한실험실,"[트리트먼트, 모든 모발용, 머릿결개선, 촉촉함(수분공급), 윤기부여, 영양공급]",False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
merged_df

,ID,상품명,특징,attribute_id
0,32792384618,샴푸바 S19 바오밥 3개 + 노워시 트리트먼트 바 1개,헤어케어세트,0
1,20621914026,다시마 샴푸 500ml + 다시마 트리트먼트 500ml 세트,헤어케어세트,0
2,29520836625,세라 화이트 머스크 2종 세트,헤어케어세트,0
3,29650571619,데저티콜라 2종 세트,헤어케어세트,0
4,20572265693,흑곡 샴푸 500ml + 트리트먼트 500ml 세트,헤어케어세트,0
...,...,...,...,...
63138,28220203522,닥터비비앤 물없이 사용하는 헤어 샴푸 물티슈 42g,"시트,티슈형",133
63139,38803883619,뉴트리언트 헤어 스팀 팩,"시트,티슈형",133
63140,41047949619,센터리 안티 옐로우 컨디셔너 200ml,보색린스,134
63141,2600008423,헤어 코팅 에센스 100ml,모발보호,135


In [32]:
attributes

,attribute_id,특징
0,0,헤어케어세트
1,1,샴푸
2,2,린스
3,3,트리트먼트
4,4,머릿결개선
...,...,...
131,131,모발탄력
132,132,"린스,컨디셔너"
133,133,"시트,티슈형"
134,134,보색린스
